# Aries Basic Controller - Openmined PryVote Community (Alice)

This notebook walks through a MVP for PryVote assuming the Openminded community will be issued an Openmined commuity credential that will enable community members to vote on basic things. The credential will be issued across a previously established connection. 

It is best run in parallel with the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) controlling the agent receiving the credential to enable voting.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)


## Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [11]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'state': 'active', 'updated_at': '2020-08-18 14:23:20.442354Z', 'invitation_mode': 'once', 'connection_id': 'd7b436ec-62e8-43e4-a25f-8336a61a845b', 'my_did': '8in2BBEsL38RFAWgWcVUfJ', 'invitation_key': 'LT3sNBxQtKqjt9d3G3qPuyBEQABkkesPXLam9MaTDT5', 'initiator': 'external', 'request_id': 'e4a5d891-b66a-4615-8b49-7c0d463fa258', 'accept': 'manual', 'their_label': 'Bob', 'created_at': '2020-08-18 14:23:10.304946Z', 'routing_state': 'none', 'their_did': 'Fm38uQRi7fekTgH9VuDpU3'}]
Connection : {'state': 'active', 'updated_at': '2020-08-18 14:23:20.442354Z', 'invitation_mode': 'once', 'connection_id': 'd7b436ec-62e8-43e4-a25f-8336a61a845b', 'my_did': '8in2BBEsL38RFAWgWcVUfJ', 'invitation_key': 'LT3sNBxQtKqjt9d3G3qPuyBEQABkkesPXLam9MaTDT5', 'initiator': 'external', 'request_id': 'e4a5d891-b66a-4615-8b49-7c0d463fa258', 'accept': 'manual', 'their_label': 'Bob', 'created_at': '2020-08-18 14:23:10.304946Z', 'routing_state': 'none', 'their_did': 'Fm38uQRi7fekTgH9VuDpU3'}
Active Connect

## Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [5]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_community-member1"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "community_member"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1


## Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [6]:
response = await agent_controller.definitions.write_cred_def(schema_id)
print(response)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)
# cred_def_id = 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default'

{'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default'}
PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


**Note: You should be able to see both schema and definition transactions on the local network [here](http://localhost:9000)**

## Populate the Attribues to Issue to Openmind-pryvote-voter (Bob)

The schema defines two attributes: name and community_member. Both must be populated with attribute values that Openmined Community (Alice) wishes to issue in a credential to Openmind-pryvote-voter (Bob). To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [7]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "community_member", "value": "1"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## Send Credential

This sends a credential to a holder Openmind-pryvote-voter (Bob), and automates the rest of the protocol. This tutorial works best if you have initialised the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) and registered the listener for the issue_credential webhook.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [12]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
#extract issuer DID for proof request requirements
issuer_did = record['credential_proposal_dict']['issuer_did']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]}, 'connection_id': 'd7b436ec-62e8-43e4-a25f-8336a61a845b', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'schema_name': 'open_mined_community-member1', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Credential exchange e4205e64-af5f-4dc9-bdfe-996c9e519769, role: issuer, state: offer_sent
Handle Credentials
Credential exchange e4205e64-af5f-4dc9-bdfe-996c9e519769, role: issuer, state: offer_sent
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [13]:
response = await agent_controller.issuer.get_records()
print(response['results'])

[{'state': 'offer_sent', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'connection_id': 'd7b436ec-62e8-43e4-a25f-8336a61a845b', 'auto_issue': True, 'thread_id': '29bb00d4-d0ff-456d-ac05-872b486c15d9', 'auto_offer': False, 'role': 'issuer', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '99133546511852264125069053146389797344738821522860595195262741593659359789448', 'xz_cap': '509702307889563686829995636834521986762207159714283186434525823851724089762468557227391492072905095069554853638302389911834297806659462618305715572617963000043531325521228580816028533745345840718438235339688377862597749347981287755948507900004647557848085131611489468283552823101621612608026310723175764733851523833418629580465255129474343140644559565210918168312376688202173156789123893605794245599595521457070939394429389409190212656136890969554017331

In [14]:
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange e4205e64-af5f-4dc9-bdfe-996c9e519769, role: issuer, state: offer_sent
Handle Credentials
Credential exchange e4205e64-af5f-4dc9-bdfe-996c9e519769, role: issuer, state: request_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange e4205e64-af5f-4dc9-bdfe-996c9e519769, role: issuer, state: credential_issued
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange e4205e64-af5f-4dc9-bdfe-996c9e519769, role: issuer, state: credential_acked
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?).

In [13]:
response = await agent_controller.issuer.remove_record(record_id)
print(response)

{}


In [29]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': []}


# Send Proof Request

Request a proof of Opemined community member to prove community membership

In [17]:
print("Request proof of Openmined Community from Openmined PryVote Voter (Bob)")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

req_attrs = [
    {"name": "name", "restrictions": [{"issuer_did": issuer_did}]},
#     {"name": "community_member", "restrictions": [{"issuer_did": issuer_did}]},
]

# if revocation:
#     req_attrs.append(
#         {
#             "name": "degree",
#             "restrictions": [{"issuer_did": agent.did}],
#             "non_revoked": {"to": int(time.time() - 1)},
#         },
#     )
# else:
#     req_attrs.append(
#         {"name": "vote", "restrictions": [{"issuer_did": issuer_did}]}
#     )

if SELF_ATTESTED:
    # test self-attested claims
    req_attrs.append({"name": "community_vote"},)
    
#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "community_member",
        "p_type": ">=",
        "p_value": 1,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Openmined Community Member",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid": 
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid": 
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}
    
proof_request = indy_proof_request
# exchange_tracing_id = exchange_tracing
# proof_request_web_request = {
#     "connection_id": agent.connection_id,
#     "proof_request": indy_proof_request,
#     "trace": exchange_tracing,
# }

Request proof of Openmined Community from Openmined PryVote Voter (Bob)


In [18]:
response = await agent_controller.proofs.send_request(connection_id, proof_request, "", exchange_tracing)
print(response)

{'state': 'request_sent', 'role': 'verifier', 'updated_at': '2020-08-18 14:32:18.394490Z', 'connection_id': 'd7b436ec-62e8-43e4-a25f-8336a61a845b', 'auto_present': False, 'presentation_request': {'name': 'Proof of Openmined Community Member', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_community_vote_uuid': {'name': 'community_vote'}}, 'requested_predicates': {'0_community_member_GE_uuid': {'name': 'community_member', 'p_type': '>=', 'p_value': 1, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '336136574308128896552538'}, 'initiator': 'self', 'created_at': '2020-08-18 14:32:18.394490Z', 'thread_id': '6d7b81a3-58c6-48ed-af3e-aa8900c2197c', 'trace': False, 'presentation_exchange_id': '1db7aff8-3f05-4088-a1b8-4aa26ee90cbc'}


# Verify Proof Presentation

In [19]:
verify = await agent_controller.proofs.verify_presentation('1db7aff8-3f05-4088-a1b8-4aa26ee90cbc')
print(verify)

{'state': 'verified', 'role': 'verifier', 'updated_at': '2020-08-18 14:36:16.014690Z', 'connection_id': 'd7b436ec-62e8-43e4-a25f-8336a61a845b', 'auto_present': False, 'presentation_request': {'name': 'Proof of Openmined Community Member', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_community_vote_uuid': {'name': 'community_vote'}}, 'requested_predicates': {'0_community_member_GE_uuid': {'name': 'community_member', 'p_type': '>=', 'p_value': 1, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '336136574308128896552538'}, 'initiator': 'self', 'verified': 'false', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'name': '93006290325627508022776103386395994712401809437930957652111221015872244345185'}, 'a_prime': '4465728769011958402707285722482421718883038790916664292695830899210627979312437491005361699241517658986296151909654418664420

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

There are a few api calls not covered in this tutorial so far. Mostly around credential revocation.

In [20]:
response = await agent_controller.terminate()
print(response)

None
